In [1]:
import mrsqm
import numpy as np
import pandas as pd
from sklearn import metrics
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import timeit

ModuleNotFoundError: No module named 'mrsqm'

In [2]:
!pip install -e ../mrsqm/

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///C:/Users/cri98/Documents/cri98tj/Other%20classificators%20prototypes/mrsqm/mrsqm


    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\cri98\\Documents\\cri98tj\\Other classificators prototypes\\mrsqm\\mrsqm\\setup.py'"'"'; __file__='"'"'C:\\Users\\cri98\\Documents\\cri98tj\\Other classificators prototypes\\mrsqm\\mrsqm\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' develop --no-deps --user --prefix=


  Running setup.py develop for mrsqm


         cwd: C:\Users\cri98\Documents\cri98tj\Other classificators prototypes\mrsqm\mrsqm\
    Complete output (123 lines):
    running develop
    C:\ProgramData\Anaconda3\lib\site-packages\setuptools\command\easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
      warnings.warn(
    C:\ProgramData\Anaconda3\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
      warnings.warn(
    running egg_info
    creating mrsqm.egg-info
    writing mrsqm.egg-info\PKG-INFO
    writing dependency_links to mrsqm.egg-info\dependency_links.txt
    writing top-level names to mrsqm.egg-info\top_level.txt
    writing manifest file 'mrsqm.egg-info\SOURCES.txt'
    reading manifest file 'mrsqm.egg-info\SOURCES.txt'
    writing manifest file 'mrsqm.egg-info\SOURCES.txt'
    running build_ext
    building '

    C:\Users\cri98\Documents\cri98tj\Other classificators prototypes\mrsqm\mrsqm\sax_converter.h(187): warning C4365: 'argomento': conversione da 'int' a 'const unsigned __int64'. Errore di corrispondenza tra signed e unsigned.
    C:\Users\cri98\Documents\cri98tj\Other classificators prototypes\mrsqm\mrsqm\sax_converter.h(191): warning C4365: 'argomento': conversione da 'int' a 'const unsigned __int64'. Errore di corrispondenza tra signed e unsigned.
    C:\Users\cri98\Documents\cri98tj\Other classificators prototypes\mrsqm\mrsqm\sax_converter.h(196): warning C4365: 'argomento': conversione da 'int' a 'const unsigned __int64'. Errore di corrispondenza tra signed e unsigned.
    C:\Users\cri98\Documents\cri98tj\Other classificators prototypes\mrsqm\mrsqm\sax_converter.h(200): warning C4365: 'argomento': conversione da 'int' a 'const unsigned __int64'. Errore di corrispondenza tra signed e unsigned.
    C:\Users\cri98\Documents\cri98tj\Other classificators prototypes\mrsqm\mrsqm\sax_con

In [ ]:
# code to read arff data
# Source: https://github.com/alan-turing-institute/sktime/blob/main/sktime/utils/data_io.py

def load_from_arff_to_dataframe(

    full_file_path_and_name,
    has_class_labels=True,
    return_separate_X_and_y=True,
    replace_missing_vals_with="NaN",
):
    """Load data from a .ts file into a Pandas DataFrame.
    Parameters
    ----------
    full_file_path_and_name: str
        The full pathname of the .ts file to read.
    has_class_labels: bool
        true then line contains separated strings and class value contains
        list of separated strings, check for 'return_separate_X_and_y'
        false otherwise.
    return_separate_X_and_y: bool
        true then X and Y values should be returned as separate Data Frames (
        X) and a numpy array (y), false otherwise.
        This is only relevant for data.
    replace_missing_vals_with: str
       The value that missing values in the text file should be replaced
       with prior to parsing.
    Returns
    -------
    DataFrame, ndarray
        If return_separate_X_and_y then a tuple containing a DataFrame and a
        numpy array containing the relevant time-series and corresponding
        class values.
    DataFrame
        If not return_separate_X_and_y then a single DataFrame containing
        all time-series and (if relevant) a column "class_vals" the
        associated class values.
    """
    
    instance_list = []
    class_val_list = []

    data_started = False
    is_multi_variate = False
    is_first_case = True

    # Parse the file
    # print(full_file_path_and_name)
    with open(full_file_path_and_name, "r", encoding="utf-8") as f:
        for line in f:

            if line.strip():
                if (
                    is_multi_variate is False
                    and "@attribute" in line.lower()
                    and "relational" in line.lower()
                ):
                    is_multi_variate = True

                if "@data" in line.lower():
                    data_started = True
                    continue

                # if the 'data tag has been found, the header information
                # has been cleared and now data can be loaded
                if data_started:
                    line = line.replace("?", replace_missing_vals_with)

                    if is_multi_variate:
                        if has_class_labels:
                            line, class_val = line.split("',")
                            class_val_list.append(class_val.strip())
                        dimensions = line.split("\\n")
                        dimensions[0] = dimensions[0].replace("'", "")

                        if is_first_case:
                            for _d in range(len(dimensions)):
                                instance_list.append([])
                            is_first_case = False

                        for dim in range(len(dimensions)):
                            instance_list[dim].append(
                                pd.Series(
                                    [float(i) for i in dimensions[dim].split(",")]
                                )
                            )

                    else:
                        if is_first_case:
                            instance_list.append([])
                            is_first_case = False

                        line_parts = line.split(",")
                        if has_class_labels:
                            instance_list[0].append(
                                pd.Series(
                                    [
                                        float(i)
                                        for i in line_parts[: len(line_parts) - 1]
                                    ]
                                )
                            )
                            class_val_list.append(line_parts[-1].strip())
                        else:
                            instance_list[0].append(
                                pd.Series(
                                    [float(i) for i in line_parts[: len(line_parts)]]
                                )
                            )

    x_data = pd.DataFrame(dtype=np.float32)
    for dim in range(len(instance_list)):
        x_data["dim_" + str(dim)] = instance_list[dim]

    if has_class_labels:
        if return_separate_X_and_y:
            return x_data, np.asarray(class_val_list)
        else:
            x_data["class_vals"] = pd.Series(class_val_list)

    return x_data

# code to visualize saliency map
# Source: https://github.com/mlgig/explanation4tsc/blob/master/3.%20Compare%20Metrics%20and%20Draw%20Figures.ipynb
def plot_time_series_with_color(ts, weight, save = False):   
    cas = weight
    
    def transform(X):
        ma,mi = np.max(X), np.min(X)
        X = (X - mi)/(ma-mi)
        return X*100
    cas = transform(cas)

    max_length1, max_length2 = len(weight),10000 #
    x1 = np.linspace(0,max_length1,num = max_length1)
    x2 = np.linspace(0,max_length1,num = max_length2)
    y1 = ts
    f = interp1d(x1, y1)

    fcas = interp1d(x1, cas)
    cas = fcas(x2)

    plt.figure(figsize = (5,3.5))
    plt.scatter(x2,f(x2), c = cas, cmap = 'jet', marker='.', s= 1,vmin=0,vmax = 100)    
    plt.colorbar()
    if save: plt.savefig('imgout/SM_%s_%s_Class%d_idx%d.png' %(dataset, explanation_method, label[i],i))


: 

: 

: 

: 

In [ ]:
dataset="Coffee"
X_train,y_train = load_from_arff_to_dataframe("data/" + dataset + "/" + dataset + "_TRAIN.arff")
X_test,y_test = load_from_arff_to_dataframe("data/" + dataset + "/" + dataset + "_TEST.arff")

print(dataset)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

X_train

: 

In [ ]:
!ls ../../../examples/Animals\ Dataset/data/

: 

In [ ]:
df = pd.read_csv("../../../examples/Animals Dataset/data/animals_preapred.zip")

df

: 

In [ ]:
"""df.c1 = 1
df.c2 = 2
df["class"] = 'B'"""

: 

In [ ]:
def from_2d_array_to_nested(
    X, index=None, columns=None, time_index=None, cells_as_numpy=False
):
    """Convert tabular pandas DataFrame with only primitives in cells into
    nested pandas DataFrame with a single column.
    Parameters
    ----------
    X : pd.DataFrame
    cells_as_numpy : bool, default = False
        If True, then nested cells contain NumPy array
        If False, then nested cells contain pandas Series
    index : array-like, shape=[n_samples], optional (default = None)
        Sample (row) index of transformed DataFrame
    time_index : array-like, shape=[n_obs], optional (default = None)
        Time series index of transformed DataFrame
    Returns
    -------
    Xt : pd.DataFrame
        Transformed DataFrame in nested format
    """

    if (time_index is not None) and cells_as_numpy:
        raise ValueError(
            "`Time_index` cannot be specified when `return_arrays` is True, "
            "time index can only be set to "
            "pandas Series"
        )
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()

    container = np.array if cells_as_numpy else pd.Series

    # for 2d numpy array, rows represent instances, columns represent time points
    n_instances, n_timepoints = X.shape

    if time_index is None:
        time_index = np.arange(n_timepoints)
    kwargs = {"index": time_index}

    Xt = pd.DataFrame(
        pd.Series([container(X[i, :], **kwargs) for i in range(n_instances)])
    )
    if index is not None:
        Xt.index = index
    if columns is not None:
        Xt.columns = columns
    return Xt

: 

In [ ]:
df["pos"] = df.groupby(by=["tid"]).cumcount()

df_pivot_c1 = df.pivot(index=['tid'], columns="pos", values='c1')#.fillna(0)
df_pivot_c2 = df.pivot(index=['tid'], columns="pos", values='c2')#.fillna(0)

df_pivot_c1 = from_2d_array_to_nested(df_pivot_c1)
df_pivot_c2 = from_2d_array_to_nested(df_pivot_c2)

#df_pivot_c1 = df_pivot_c1.fillna(0.0).astype('float64')
#df_pivot_c2 = df_pivot_c2.fillna(0.0).astype('float64')

: 

In [ ]:
df_pivot = pd.DataFrame()
df_pivot["dim_0"] = df_pivot_c1[0]
df_pivot["dim_1"] = df_pivot_c2[0]

df_pivot

: 

In [ ]:
y = df.groupby(by=["tid"]).max()["class"]

: 

In [ ]:
df_pivot.to_pickle("df_pivot")
y.to_pickle("y")

: 

: 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_pivot, y, test_size=0.3)

: 

In [ ]:
print(len(X_train) )
len(y_train)

: 

## Train with MrSQMClassifier

In [ ]:
def from_nested_to_2d_array(X, return_numpy=False):
    """Convert nested Panel to 2D numpy Panel.

    Convert nested pandas DataFrame or Series with NumPy arrays or
    pandas Series in cells into tabular
    pandas DataFrame with primitives in cells, i.e. a data frame with the
    same number of rows as the input data and
    as many columns as there are observations in the nested series. Requires
    series to be have the same index.

    Parameters
    ----------
    X : nested pd.DataFrame or nested pd.Series
    return_numpy : bool, default = False
        - If True, returns a NumPy array of the tabular data.
        - If False, returns a pandas DataFrame with row and column names.

    Returns
    -------
     Xt : pandas DataFrame
        Transformed DataFrame in tabular format
    """
    # TODO does not handle dataframes with nested series columns *and*
    #  standard columns containing only primitives

    # convert nested data into tabular data
    if isinstance(X, pd.Series):
        Xt = np.array(X.tolist())

    elif isinstance(X, pd.DataFrame):
        try:
            Xt = np.hstack([X.iloc[:, i].tolist() for i in range(X.shape[1])])

        # except strange key error for specific case
        except KeyError:
            if (X.shape == (1, 1)) and (X.iloc[0, 0].shape == (1,)):
                # in fact only breaks when an additional condition is met,
                # namely that the index of the time series of a single value
                # does not start with 0, e.g. pd.RangeIndex(9, 10) as is the
                # case in forecasting
                Xt = X.iloc[0, 0].values
            else:
                raise

        if Xt.ndim != 2:
            raise ValueError(
                "Tabularization failed, it's possible that not "
                "all series were of equal length"
            )

    else:
        raise ValueError(
            f"Expected input is pandas Series or pandas DataFrame, "
            f"but found: {type(X)}"
        )

    if return_numpy:
        return Xt

    Xt = pd.DataFrame(Xt)

    # create column names from time index
    if X.ndim == 1:
        time_index = (
            X.iloc[0].index
            if hasattr(X.iloc[0], "index")
            else np.arange(X.iloc[0].shape[0])
        )
        columns = [f"{X.name}__{i}" for i in time_index]

    else:
        columns = []
        for colname, col in X.items():
            time_index = (
                col.iloc[0].index
                if hasattr(col.iloc[0], "index")
                else np.arange(col.iloc[0].shape[0])
            )
            columns.extend([f"{colname}__{i}" for i in time_index])

    Xt.index = X.index
    Xt.columns = columns
    return Xt



: 

In [ ]:
import warnings
from tqdm.auto import tqdm

from tslearn.piecewise import SymbolicAggregateApproximation

def generateAlphabet(dim):
    return [(97+i).to_bytes(1, byteorder='big')for i in range(dim)] #97 --> 'a'

def test(conf, ts_series):
    ts_series = ts_series.dropna()
    #esempio conf: {'method': 'sax', 'window': 64, 'word': 16, 'alphabet': 3, 'dilation': 1}
    #print(conf, flush=True)

    sax = SymbolicAggregateApproximation(n_segments=conf["word"], alphabet_size_avg=conf["alphabet"], scale=True)
    alphabet = generateAlphabet(conf["alphabet"])
    result = b''
    for i in [0]+list(range(1, len(ts_series)-conf["window"])):
        if i != 0:
            result += b' '
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            X_sax = sax.fit_transform([ts_series.values[i:i+conf["window"]]])
        for char in X_sax[0]:
            result += alphabet[char[0]]

    return result

def MY_transform_time_series(self, ts_x):

        multi_tssr = []

        #ts_x_array = from_nested_to_2d_array(ts_x).values

        if not self.config:
            self.config = []

            min_ws = 16
            min_len = max_len = len(ts_x.iloc[0, 0])
            for a in ts_x.iloc[:, 0]:
                min_len = min(min_len, len(a))
                max_len = max(max_len, len(a))
            max_ws = (min_len + max_len) // 2

            pars = self.create_pars(min_ws, max_ws, self.nsax, random_sampling=True, is_sfa=False)
            for p in pars:
                self.config.append(
                    {'method': 'sax', 'window': p[0], 'word': p[1], 'alphabet': p[2],
                     # 'dilation': np.int32(2 ** np.random.uniform(0, np.log2((min_len - 1) / (p[0] - 1))))})
                     'dilation': 1})

            pars = self.create_pars(min_ws, max_ws, self.nsfa, random_sampling=True, is_sfa=True)
            for p in pars:
                self.config.append(
                    {'method': 'sfa', 'window': p[0], 'word': p[1], 'alphabet': p[2], 'normSFA': False,
                     'normTS': self.sfa_norm
                     })

        for cfg in tqdm(self.config, desc="converting to sax"):
            for i in range(ts_x.shape[1]):
                tssr = []

                if cfg['method'] == 'sax':  # convert time series to SAX
                    #ps = PySAX(cfg['window'], cfg['word'], cfg['alphabet'], cfg['dilation'])
                    for ts in ts_x.iloc[:, i]:
                        sr = test(cfg, ts)#ps.timeseries2SAXseq(ts)
                        if sr == b'': continue
                        tssr.append(sr)
                elif cfg['method'] == 'sfa':
                    pass
                multi_tssr.append(tssr)

        new_multi_tssr = []

        i = 0
        for conf_lat, conf_lon in tqdm(zip(multi_tssr[0::2], multi_tssr[1::2]), desc="converting to trajectory-sax",
                                       total=len(multi_tssr[::2])):
            i += 1
            new_tssr = []
            j = 0

            
            for k in range(len(conf_lat)):  #scorro le varie serie temporali
                #print(f"{i}-{k}, len={len(conf_lat)} {len(conf_lon)}  len_k={len(conf_lat[k])} {len(conf_lon[k])}")
                seq = b''
                c = b'a'
                dizionario = {}
                
                #test con riordinamento
                s_lat_list = conf_lat[k]
                s_lon_list = conf_lon[k]

                s_lat_list.sort()
                s_lon_list.sort()

                for sym1, sym2 in zip(s_lat_list, s_lat_list):
                    if key not in dizionario:
                        c = (int.from_bytes(c, byteorder='big') + 1).to_bytes(1, byteorder='big')
                        dizionario[key] = c


                for sym1, sym2 in zip(conf_lat[k], conf_lon[k]):
                    j += 1

                    if sym1 == 32:
                        seq += b' '
                        continue

                    key = (sym1, sym2)
                    if key not in dizionario:
                        print("ERRORE")
                    seq += dizionario[key]
                new_tssr.append(seq)
            new_multi_tssr.append(new_tssr)

        return multi_tssr, new_multi_tssr



: 

In [ ]:
from datetime import datetime

from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def compute_measures(test, pred):
    return (accuracy_score(test, pred), precision_score(test, pred, average="micro"), f1_score(test, pred, average="micro"), recall_score(test, pred, average="micro"))

res = []
time = []

for i in tqdm(range(1, 15)):
    clf = mrsqm.MrSQMClassifier(MY_transform_time_series, nsax=int(1.5**i), nsfa=0, random_state=42)
    start = datetime.now()
    clf.fit(X_train,y_train)
    stop = start - datetime.now()
    y_pred = clf.predict(X_test)
    print(metrics.accuracy_score(y_test, y_pred))
    
    res.append(compute_measures(y_test, y_pred))
    
    time.append(stop.total_seconds()*1000)
    

#try:
#    clf.fit(X_train,y_train)
#except Exception as e: #raise Exception(X, mr_seqs, self.sequences)
#    X = e.args[0]
#    mr_seqs_old = e.args[1]
#    mr_seqs = e.args[2]
#    sequences = e.args[3]
#    print("TUTTO OK")

: 

In [ ]:
df_res_rig = pd.DataFrame(res, columns=["acc", "prec", "f1", "recall"])

df_res_rig["t"] = time

df_res_rig.t *= -1

df_res_rig["n"] = [int(1.5**x) for x in range(1, 13)]

df_res_rig.to_csv("Test MrSQM Animals.csv", index=None)

: 

In [ ]:
df_res_rig

: 

: 

In [ ]:
matrix = np.zeros(())

: 

: 

: 

: 

: 

In [ ]:
len(X)

: 

In [ ]:
len(y)

: 

In [ ]:
mr_seqs

: 

In [ ]:
mr_seqs_old

: 

In [ ]:
mr_seqs[2]

: 

In [ ]:
mr_seqs_old[0][0]

: 

In [ ]:
clf.config

: 

__mr\_seql__: lista di liste

Ogni lista rappresenta una configurazione (tot: 8*num_dim liste).

Ogni lista di "primo livello" è composta da altre 28 liste (= al numero di istanze)

Queste liste di "secondo livello" contengono un array di byte di lunghezza variabile.
- la lunghezza di questo array dipende dalla TS? (sono tutte lunghe 286 valori)

## Test the model

In [ ]:
clf.filters

: 

In [ ]:
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

: 

: 

In [ ]:
clf.train_x

: 

: 

: 

: 

: 

: 

: 